In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()

In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-194878
Azure region: southcentralus
Subscription id: 61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30
Resource group: aml-quickstarts-194878


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "bogicluster1"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.


try:
    my_cluster=ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing cluster. Use it.")
except:
    compute_config=AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=4)
    my_cluster=ComputeTarget.create(workspace=ws, name=cluster_name, provisioning_configuration=compute_config )
    my_cluster.wait_for_completion(show_output=True)

InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
import os
os.getcwd()

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/cluster-basic/code/Users/odl_user_194878/udacity_azure_project1-master'

In [19]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler

ps = RandomParameterSampling(parameter_space={'max_iter': choice(10,50,100,150), 'C': uniform(0.2,2)})

# Specify a Policy
policy = BanditPolicy(slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='/mnt/batch/tasks/shared/LS_root/mounts/clusters/cluster-basic/code/Users/odl_user_194878/udacity_azure_project1-master',
    script='./train.py',
    compute_target=my_cluster,
    environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config =HyperDriveConfig(hyperparameter_sampling=ps,primary_metric_name="Accuracy",primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
policy=policy,run_config=src, max_total_runs=5)

In [20]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

experiment=Experiment(ws,'myexperiment')
hyperdrive_run=experiment.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [28]:
import joblib
# Get your best run and save the model from that run.

best_run=hyperdrive_run.get_best_run_by_primary_metric()
joblib.dump(best_run.id,'best_model.joblib')

['best_model.joblib']

In [27]:
print(best_run.id)


HD_d0a922dc-d8ab-4828-a489-e148b11de813_4


In [29]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [30]:
from train import clean_data
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=50)

In [31]:
x_train['target_var']=y_train


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [32]:
x_train

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,target_var
25879,47,1,0,1,0,8,2,122,1,999,...,0,1,0,0,0,0,0,0,0,0
4183,36,0,0,0,1,8,2,35,2,999,...,0,0,0,0,0,0,0,1,0,0
19659,59,1,0,1,0,8,4,152,2,999,...,0,0,0,1,0,0,0,0,0,0
23342,47,1,0,1,1,6,5,150,2,999,...,1,0,1,0,0,0,0,0,0,0
32835,39,1,0,0,0,6,1,135,2,999,...,1,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19406,45,1,0,0,0,5,5,144,3,999,...,0,0,0,1,0,0,0,0,0,0
8559,25,0,0,1,0,5,1,52,9,999,...,1,0,0,0,1,0,0,0,0,0
32022,46,0,0,1,0,5,1,18,5,999,...,0,0,1,0,0,0,0,0,0,0
22637,54,0,0,0,0,6,4,170,1,999,...,1,1,0,0,0,0,0,0,0,0


In [33]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data= x_train,
    label_column_name='target_var',
    n_cross_validations= 5)

In [34]:
# Submit your automl run

experiment_automl=Experiment(ws,'AutoMl_experiment')
run_automl=experiment_automl.submit(config=automl_config,show_output=True)

No run_configuration provided, running on local with default configuration
Running in the active local environment.


2022-05-08:09:24:35,336 INFO     [modeling_bert.py:226] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
2022-05-08:09:24:35,915 INFO     [modeling_xlnet.py:339] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


Experiment,Id,Type,Status,Details Page,Docs Page
AutoMl_experiment,AutoML_a9312d47-6d82-4d1c-95ac-6794ff77cf55,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one cl

2022-05-08:09:58:37,85 INFO     [explanation_client.py:334] Using default datastore for uploads


In [35]:
# Retrieve and save your best automl model.

best_run,fitted_model=run_automl.get_output()
joblib.dump(fitted_model,'automml_best_model.joblib')


['automml_best_model.joblib']

In [36]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
AutoMl_experiment,AutoML_a9312d47-6d82-4d1c-95ac-6794ff77cf55_28,,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [38]:
#delete_compute('bogicluster1')
AmlCompute.delete(my_cluster)

Current provisioning state of AmlCompute is "Deleting"

